## Import libs, set paths and load params

In [1]:
import os
import numpy as np
import pandas as pd
import sys
sys.path.insert(0, "../src")

import auxilary_functions as f

## Convert raw data to adjacency matrices

In [2]:
#ecoli
df = f.read_ecoli_network("../data/ori/ecoli/network_tf_gene.txt")
edges = df[[1, 3]]
edges.columns = ["tf", "tg"]
edges.tf = edges.tf.apply(lambda x: x.lower())
edges.tg = edges.tg.apply(lambda x: x.lower())
nodes = sorted(np.unique(np.concatenate((edges.tf.unique(), edges.tg.unique()))))
nodes = pd.DataFrame(data=range(len(nodes)), index=nodes, columns=["idx"])
edges_ = edges.join(nodes, on="tf").join(nodes, on="tg", lsuffix="_tf", rsuffix="_tg")
np_edges = edges_[["idx_tg", "idx_tf"]].values
interaction_matrix_ecoli = f.build_Tnet(np_edges, len(nodes))

In [3]:
#yeast
edges = pd.read_csv("../data/ori/yeast/tnet.txt", sep="\t")
nodes = sorted(np.unique(np.concatenate((edges.Tf.unique(), edges.Tg.unique()))))
nodes = pd.DataFrame(data=range(len(nodes)), index=nodes, columns=["idx"])
edges_ = edges.join(nodes, on="Tf").join(nodes, on="Tg", lsuffix="_tf", rsuffix="_tg")
np_edges = edges_[["idx_tg", "idx_tf"]].values
interaction_matrix_yeast = f.build_Tnet(np_edges, len(nodes))

In [4]:
#mouse
edges = pd.read_csv(("../data/ori/trrust/trrust_rawdata.mouse.tsv"), sep="\t", header=None)[[0, 1]]
edges.columns = ["Tf", "Tg"]
nodes = sorted(np.unique(np.concatenate((edges.Tf.unique(), edges.Tg.unique()))))
nodes = pd.DataFrame(data=range(len(nodes)), index=nodes, columns=["idx"])
edges_ = edges.join(nodes, on="Tf").join(nodes, on="Tg", lsuffix="_tf", rsuffix="_tg")
np_edges = edges_[["idx_tg", "idx_tf"]].values
interaction_matrix_mouse = f.build_Tnet(np_edges, len(nodes))

In [5]:
#human
edges = pd.read_csv(("../data/ori/trrust/trrust_rawdata.human.tsv"), sep="\t", header=None)[[0, 1]]
edges.columns = ["Tf", "Tg"]
nodes = sorted(np.unique(np.concatenate((edges.Tf.unique(), edges.Tg.unique()))))
nodes = pd.DataFrame(data=range(len(nodes)), index=nodes, columns=["idx"])
edges_ = edges.join(nodes, on="Tf").join(nodes, on="Tg", lsuffix="_tf", rsuffix="_tg")
np_edges = edges_[["idx_tg", "idx_tf"]].values
interaction_matrix_human = f.build_Tnet(np_edges, len(nodes))

## Create table

In [6]:
cfg = f.get_actual_parametrization("../src/config.json")
table1 = pd.DataFrame([f.collect_topological_parameters(cfg,interaction_matrix_ecoli, 'e.coli'),
                      f.collect_topological_parameters(cfg,interaction_matrix_yeast, 'yeast'),
                      f.collect_topological_parameters(cfg,interaction_matrix_mouse, 'mouse'),
                      f.collect_topological_parameters(cfg,interaction_matrix_human, 'human')], 
                  columns = ['ffl_node_participation' , 'sparsity', 'in-degree' , 'out-degree', 'organism']) 

table1.to_csv("../results/tables/table1.tsv", sep="\t", index=False)
table1

,ffl_node_participation,sparsity,in-degree,out-degree,organism
0,1392,2.328,1.106,1.222,e.coli
1,3370,2.899,1.421,1.477,yeast
2,1956,2.643,1.274,1.369,mouse
3,4152,2.944,1.364,1.580,human
